In [1]:
import tensorflow as tf
import numpy as np
import datetime
import random
from sklearn.model_selection import train_test_split

from data.generator import generateDataSet, saveDataSet, timedDataSet

2022-10-26 10:00:50.991125: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-26 10:00:51.117079: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/home/watcherwhale/.local/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [25]:
input1, output1, mean1, std1 = generateDataSet("./datasets/raw/logs_WT.csv", 
                                "./datasets/raw/Resultaten_WT.csv", 
                                datetime.datetime(2020, 9, 15, 0, 0).timestamp(), 
                                datetime.datetime(2021, 2, 10, 0, 0).timestamp(),
                                50)
input2, output2, mean2, std2 = generateDataSet("./datasets/raw/logs_DN.csv", 
                                "./datasets/raw/Resultaten_DN.csv", 
                                datetime.datetime(2020, 9, 15, 0, 0).timestamp(), 
                                datetime.datetime(2021, 2, 10, 0, 0).timestamp(),
                                50)

input3, output3, mean3, std3 = generateDataSet("./datasets/raw/logs_RS.csv", 
                                "./datasets/raw/Resultaten_DN.csv", 
                                datetime.datetime(2021, 2, 1, 0, 0).timestamp(), 
                                datetime.datetime(2021, 6, 30, 0, 0).timestamp(),
                                50)
seed1 = random.randrange(0,100)
seed2 = random.randrange(0,100)
seed3 = random.randrange(0,100)

random.Random(seed1).shuffle(input1)
random.Random(seed1).shuffle(output1)

random.Random(seed2).shuffle(input2)
random.Random(seed2).shuffle(output2)

random.Random(seed3).shuffle(input3)
random.Random(seed3).shuffle(output3)

input = []
output = []

input.extend(input1[0:10000])
output.extend(output1[0:10000])

input.extend(input2[0:10000])
output.extend(output2[0:10000])

input.extend(input3[0:10000])
output.extend(output3[0:10000])

x = np.array(input)
y = np.array(output)

In [11]:
print(mean1)
print(std1)

2.5502958579881656
0.6048251994878409


In [26]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

print(len(x_train))
print(len(x_test))

21337
7113


In [27]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(6, 1)),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(512,  activation='relu'),
    tf.keras.layers.Dense(64,   activation='relu'),
    tf.keras.layers.Dense(32,   activation='relu'),
    tf.keras.layers.Dense(16,   activation='relu'),
    tf.keras.layers.Dense(1),
])

model.compile(optimizer=tf.optimizers.Adam(), loss='huber', metrics=['mse', 'mae', 'mape'])
model.fit(x_train, y_train, epochs=50)

Epoch 1/50
667/667 [==============================] - 25s 37ms/step - loss: 0.4686 - mse: 1.0582 - mae: 0.8631 - mape: 119.6614
Epoch 2/50
667/667 [==============================] - 25s 37ms/step - loss: 0.4511 - mse: 0.9874 - mae: 0.8416 - mape: 112.3163
Epoch 3/50
667/667 [==============================] - 30s 45ms/step - loss: 0.4483 - mse: 0.9844 - mae: 0.8408 - mape: 116.1515
Epoch 4/50
667/667 [==============================] - 27s 41ms/step - loss: 0.4439 - mse: 0.9720 - mae: 0.8290 - mape: 110.3909
Epoch 5/50
667/667 [==============================] - 27s 41ms/step - loss: 0.4387 - mse: 0.9641 - mae: 0.8264 - mape: 116.1938
Epoch 6/50
667/667 [==============================] - 27s 40ms/step - loss: 0.4320 - mse: 0.9466 - mae: 0.8152 - mape: 115.1421
Epoch 7/50
667/667 [==============================] - 31s 46ms/step - loss: 0.4296 - mse: 0.9464 - mae: 0.8092 - mape: 116.5948
Epoch 8/50
667/667 [==============================] - 24s 36ms/step - loss: 0.4232 - mse: 0.9271 - mae: 

In [ ]:
model.fit(x_train, y_train, epochs=50)

Epoch 1/50
667/667 [==============================] - 24s 37ms/step - loss: 0.2880 - mse: 0.6158 - mae: 0.5533 - mape: 86.3129
Epoch 2/50
667/667 [==============================] - 25s 37ms/step - loss: 0.2907 - mse: 0.6249 - mae: 0.5579 - mape: 87.5266
Epoch 3/50
667/667 [==============================] - 26s 39ms/step - loss: 0.2874 - mse: 0.6138 - mae: 0.5542 - mape: 86.7173
Epoch 4/50
667/667 [==============================] - 27s 40ms/step - loss: 0.2874 - mse: 0.6150 - mae: 0.5564 - mape: 88.7718
Epoch 5/50
667/667 [==============================] - 31s 47ms/step - loss: 0.2846 - mse: 0.6081 - mae: 0.5498 - mape: 86.9887
Epoch 6/50
667/667 [==============================] - 27s 40ms/step - loss: 0.2872 - mse: 0.6145 - mae: 0.5552 - mape: 88.4005
Epoch 7/50
667/667 [==============================] - 28s 43ms/step - loss: 0.2912 - mse: 0.6248 - mae: 0.5606 - mape: 88.9791
Epoch 8/50
667/667 [==============================] - 30s 46ms/step - loss: 0.2877 - mse: 0.6158 - mae: 0.5536 

In [32]:
print(np.mean(np.round(model.predict(input1) * std1 + mean1) == np.round(np.array(output1) * std1 + mean1)))
print(np.mean(np.round(model.predict(input2) * std2 + mean2) == np.round(np.array(output2) * std2 + mean2)))
print(np.mean(np.round(model.predict(input3) * std3 + mean3) == np.round(np.array(output3) * std3 + mean3)))

265/265 [==============================] - 5s 19ms/step
0.9010650887573964
540/540 [==============================] - 7s 13ms/step
0.5342028985507247
540/540 [==============================] - 6s 11ms/step
0.49727536231884056
